In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import sys, os
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

from tqdm import tqdm
from scipy import interpolate as interp
from scipy import integrate
from scipy.optimize import differential_evolution
from scipy.special import xlogy

from astropy import units as u

from spectrum import *

from NucleonCouplings import *


---

# Primakoff -- No Time Bin Information

Here we illustrate the estimate of the 95% upper limit from just Primakoff emission alone, without timing information (i.e. it is integrated over)

In [11]:
ld_dir = 'Primakoff_Spectra/Spectra_Primakoff_18p6/'
Ebins, prim = load_prim(ld_dir, Eoutput = True)
tbins = load_time(ld_dir)

In [12]:
# calculate the dN/dt
prim_dndt = np.zeros(prim.shape[0])

for i in range(prim.shape[0]):
    f_prim = interp.interp1d(Ebins, prim[i])
    prim_dndt[i] = integrate.quad(f_prim, 0.025, 0.1)[0]

fp_dndt = interp.interp1d(tbins, prim_dndt)
prim_N = integrate.quad(fp_dndt, 0, 10)[0]

/tmp/ipykernel_2038313/252834630.py:6: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  prim_dndt[i] = integrate.quad(f_prim, 0.025, 0.1)[0]
/tmp/ipykernel_2038313/252834630.py:9: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhap

In [13]:
prim_Ngamma = prim_N * 5.85e-7 * (1e-11)**2
print(f"N_gamma: {prim_Ngamma:.3e}")

N_gamma: 7.711e+44


In [14]:
prim_flux = prim_Ngamma / (4 * np.pi * (51 * u.kpc).to(u.cm).value**2) * 63
print(f"Flux: {prim_flux:.3e} cm^-2 s^-1")

Flux: 1.561e-01 cm^-2 s^-1


In [15]:
print(f"Limit on gagg = {(20 / prim_flux)**(1/4) * 1e-11:.2e} GeV^-1")

Limit on gagg = 3.36e-11 GeV^-1


---

# Primakoff -- our fiducial analysis, complete with timing information

This is our fiducial analysis in 2405.19393, which includes timing information, background profiling, utilizing all energy bins, and detailed upper limit setting

In [16]:
from smm_analysis import *

In [17]:
def calc_smm_spec(LD_dir, smm_energy, convprob):
    '''
    Calculate the SMM spectrum for a range of ma and smm_energy by integrating over energy.
    Parameters:
    -----------
    LD_dir: str
        Directory containing the spectra files.
    smm_energy: array-like
        Array of SMM energies to consider in units of GeV.
    convprob: array-like
        Array of conversion probabilities as a function of energy.
    Returns:
    --------
    spec_list: list of np.ndarray
        List of spectra arrays for each SMM energy in units of dN_gamma/dt.
    '''
    Ebins, loaded = load_spec(LD_dir, Eoutput = True)
    spec_list = []
    
    for int_I in smm_energy:
        tm_spec = np.zeros(loaded.shape[0])
        
        for ti in range(loaded.shape[0]):
            photon_spec = interp.interp1d(Ebins, loaded[ti] * convprob)
            tm_spec[ti] = integrate.quad(photon_spec, *int_I)[0]
        spec_list.append(tm_spec)
        
    return spec_list

In [42]:
spec_dirs = ["Primakoff_Spectra/Spectra_Primakoff_18p6/", "Nucleon_Spectra/Spectra_18p6/Spectra_Can/", "Nucleon_Spectra/Spectra_18p6/Spectra_Cap/", "Nucleon_Spectra/Spectra_18p6/Spectra_CanCap/"]
spec_dict = {
    'Primakoff': [],
    'n_Can': [],
    'n_Cap': [],
    'n_CanCap': []
}

for pi, stype in enumerate(spec_dict.keys()):
    print(spec_dirs[pi])
    # creating relevant photon spectrum components for 3 different energy bins in SMM data
    spec_dict[stype] = np.expand_dims(calc_smm_spec(spec_dirs[pi], [[0.0041, 0.0064], [0.01, 0.025], [0.025, 0.1]], 5.85e-9), axis=1) # 3d array of SMM energy bin, axion mass array, and time bins

spec_dict['time_bins'] = tbins # units of seconds
spec_dict['E_bins'] = Ebins # units of GeV
spec_dict['ma_array'] = [1e-21] # units of GeV

# not considering pions, creating empty arrays for axion emission from pions
spec_dict['p_Can'] = np.zeros_like(spec_dict['n_Can'])
spec_dict['p_Cap'] = np.zeros_like(spec_dict['n_Can'])
spec_dict['p_CanCap'] = np.zeros_like(spec_dict['n_Can'])

data = np.loadtxt('smm_grs_data_sn1987a_oberauer93.dat', unpack=True)

Primakoff_Spectra/Spectra_Primakoff_18p6/


/tmp/ipykernel_2034869/3279990529.py:25: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  tm_spec[ti] = integrate.quad(photon_spec, *int_I)[0]


Nucleon_Spectra/Spectra_18p6/Spectra_Can/
Nucleon_Spectra/Spectra_18p6/Spectra_Cap/
Nucleon_Spectra/Spectra_18p6/Spectra_CanCap/


In [43]:
ma_idx = 0

sm = model(ma_idx, (51.4 * u.kpc).to(u.cm).value, data[0], spec_dict, prim_flag=True, brem_flag=False, pion_flag=False)
sig_ = sm.sig_model(gagg=1.0e-12)
bkg_ = sm.bkg_model
sa = analysis(data[0], data[1:], sig_, bkg_)

sig_model_norm = np.amax(sa.sig_model) #this is a normalization factor for our signal when it undergoes fitting (see smm_analysis.py)

including primakoff
1.0280000000020664 3.0760000000009313
3.0760000000009313 5.123999999999796
5.123999999999796 7.172000000002299
7.172000000002299 9.220000000001164
1.0280000000020664 3.0760000000009313
3.0760000000009313 5.123999999999796
5.123999999999796 7.172000000002299
7.172000000002299 9.220000000001164
1.0280000000020664 3.0760000000009313
3.0760000000009313 5.123999999999796
5.123999999999796 7.172000000002299
7.172000000002299 9.220000000001164


In [46]:
norm_array = np.linspace(-1e2, 1e2, 501)
fvals = np.zeros(len(norm_array))

for i in range(len(norm_array)):
    A = norm_array[i]
    fit = lambda x: sa.negLL(np.append(x, A))
    
    out = differential_evolution(fit, [[5, 10], [-1, 1], [30, 35], [-1, 1], [10, 15], [-1, 1]], init = 'sobol', maxiter = 10000,
                                 x0 = [7.77878979e+00, -1.36391545e-04,  3.28211092e+01, -1.79343614e-02, 1.22670755e+01, -9.57101247e-04])
    if out.fun > 1e10:
        out = differential_evolution(fit, [[5, 10], [-1, 1], [30, 35], [-1, 1], [10, 15], [-1, 1]], init = 'sobol', maxiter = 10000,
                                 x0 = [7.77878979e+00, -1.36391545e-04,  3.28211092e+01, -1.79343614e-02, 1.22670755e+01, -9.57101247e-04])
        print("nan issue?")
    fvals[i] = out.fun
    
    if (i % 50 == 49):
        print(int(i / 50), end=', ')

    

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 

In [47]:

# Find minimum and compute one-sided upper 95% upper limit
imin = np.argmin(fvals)
fmin = fvals[imin]
diff_f = fvals - fmin

x = norm_array[imin:]
y = diff_f[imin:]

threshold = 2.71

norm_95 = x[np.argmin(np.abs(y - threshold))]

g_ul = (norm_95 /sig_model_norm)**(1/4) * 1.0e-12

print(r"The 95% Upper Limit is (GeV$^{-1}$): ", g_ul)

The 95% Upper Limit is (GeV$^{-1}$):  2.2098461568577002e-11


---

# Now also include Bremsstrahlung Emission

With the addition of bremsstrahlung emission, we reproduce the flat limit from magnetosphere conversion in Fig. 1 of 2405.19393.

In [18]:
spec_dirs = ["Primakoff_Spectra/Spectra_Primakoff_18p6/", "Nucleon_Spectra/Spectra_18p6/Spectra_Can/", "Nucleon_Spectra/Spectra_18p6/Spectra_Cap/", "Nucleon_Spectra/Spectra_18p6/Spectra_CanCap/"]
spec_dict = {
    'Primakoff': [],
    'n_Can': [],
    'n_Cap': [],
    'n_CanCap': []
}

for pi, stype in enumerate(spec_dict.keys()):
    print(spec_dirs[pi])
    # creating relevant photon spectrum components for 3 different energy bins in SMM data
    spec_dict[stype] = np.expand_dims(calc_smm_spec(spec_dirs[pi], [[0.0041, 0.0064], [0.01, 0.025], [0.025, 0.1]], 5.85e-9), axis=1) # 3d array of SMM energy bin, axion mass array, and time bins

spec_dict['time_bins'] = tbins # units of seconds
spec_dict['E_bins'] = Ebins # units of GeV
spec_dict['ma_array'] = [1e-21] # units of GeV

# not considering pions, creating empty arrays for axion emission from pions
spec_dict['p_Can'] = np.zeros_like(spec_dict['n_Can'])
spec_dict['p_Cap'] = np.zeros_like(spec_dict['n_Can'])
spec_dict['p_CanCap'] = np.zeros_like(spec_dict['n_Can'])

data = np.loadtxt('smm_grs_data_sn1987a_oberauer93.dat', unpack=True)

Primakoff_Spectra/Spectra_Primakoff_18p6/


/tmp/ipykernel_2038313/2995142726.py:25: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  tm_spec[ti] = integrate.quad(photon_spec, *int_I)[0]


Nucleon_Spectra/Spectra_18p6/Spectra_Can/
Nucleon_Spectra/Spectra_18p6/Spectra_Cap/
Nucleon_Spectra/Spectra_18p6/Spectra_CanCap/


In [19]:
ma_idx = 0

sm = model(ma_idx, (51.4 * u.kpc).to(u.cm).value, data[0], spec_dict, prim_flag=True, brem_flag=True, pion_flag=False)
sig_ = sm.sig_model(gagg=1.0e-12)
bkg_ = sm.bkg_model
sa = analysis(data[0], data[1:], sig_, bkg_)

sig_model_norm = np.amax(sa.sig_model) #this is a normalization factor for our signal when it undergoes fitting (see smm_analysis.py)

including primakoff
including nucleon
1.0280000000020664 3.0760000000009313
3.0760000000009313 5.123999999999796
5.123999999999796 7.172000000002299
7.172000000002299 9.220000000001164
1.0280000000020664 3.0760000000009313
3.0760000000009313 5.123999999999796
5.123999999999796 7.172000000002299
7.172000000002299 9.220000000001164
1.0280000000020664 3.0760000000009313
3.0760000000009313 5.123999999999796
5.123999999999796 7.172000000002299
7.172000000002299 9.220000000001164


In [21]:
norm_array = np.linspace(-1e2, 1e2, 501)
fvals = np.zeros(len(norm_array))

bl = 50
bi = 0

for i in range(len(norm_array)):
    A = norm_array[i]
    fit = lambda x: sa.negLL(np.append(x, A))
    
    out = differential_evolution(fit, [[5, 10], [-1, 1], [30, 35], [-1, 1], [10, 15], [-1, 1]], init = 'sobol', maxiter = 10000,
                                 x0 = [7.77878979e+00, -1.36391545e-04,  3.28211092e+01, -1.79343614e-02, 1.22670755e+01, -9.57101247e-04])
    if out.fun > 1e10:
        out = differential_evolution(fit, [[5, 10], [-1, 1], [30, 35], [-1, 1], [10, 15], [-1, 1]], init = 'sobol', maxiter = 10000,
                                 x0 = [7.77878979e+00, -1.36391545e-04,  3.28211092e+01, -1.79343614e-02, 1.22670755e+01, -9.57101247e-04])
        print("nan issue?")
    fvals[i] = out.fun
    
    if (i % 50 == 49):
        print(int(i / 50), end=', ')
    
    

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 

In [23]:

# Find minimum and compute one-sided upper 95% upper limit
imin = np.argmin(fvals)
fmin = fvals[imin]
diff_f = fvals - fmin

x = norm_array[imin:]
y = diff_f[imin:]

threshold = 2.71

norm_95 = x[np.argmin(np.abs(y - threshold))]

g_ul = (norm_95 /sig_model_norm)**(1/4) * 1.0e-12

print(r'The 95% Upper Limit is (GeV$^{-1}$): ', g_ul)

#Note that this is the upper limit on Fig. 1 (solid blue) of 2405.19393

The 95% Upper Limit is (GeV$^{-1}$):  1.853908892170983e-11
